In [5]:
import scipy.constants as const
eps_from_paper = 36.0
eps_in_joules = 36 * const.Boltzmann
eps_in_kcal_per_mole = eps_in_joules * const.Avogadro / 4184
print(eps_in_kcal_per_mole)

0.07153935331106996


In [7]:
sigma_i = 3.31
sigma_j = 3.05

sigma_ij = (sigma_i + sigma_j) / 2
print(sigma_ij)

3.1799999999999997
